In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from operations import adjust_dataset, split_data, Ann




2023-12-06 15:38:02.259811: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-06 15:38:02.263451: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-06 15:38:02.296035: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 15:38:02.296060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 15:38:02.296805: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

# Implementation of ANN

In [2]:

df = pd.read_csv('docs/data/GlobalWeatherRepository.csv')
target_variable = 'temperature_celsius'
df = adjust_dataset(df, 'Stockholm', target_variable=target_variable)



In [3]:
# lets use all the data besides the last 7 days
df = df[:-7]
model = Ann(df, 'temperature_celsius')

2023-12-06 15:38:04.754330: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-06 15:38:04.754697: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10
2/2 [==============================] - 1s 13ms/step - loss: 0.8051 - mean_squared_error: 0.8051
Epoch 2/10
2/2 [==============================] - 0s 10ms/step - loss: 0.7095 - mean_squared_error: 0.7095
Epoch 3/10
2/2 [==============================] - 0s 11ms/step - loss: 0.6276 - mean_squared_error: 0.6276
Epoch 4/10
2/2 [==============================] - 0s 10ms/step - loss: 0.5743 - mean_squared_error: 0.5743
Epoch 5/10
2/2 [==============================] - 0s 7ms/step - loss: 0.5148 - mean_squared_error: 0.5148
Epoch 6/10
2/2 [==============================] - 0s 7ms/step - loss: 0.4612 - mean_squared_error: 0.4612
Epoch 7/10
2/2 [==============================] - 0s 8ms/step - loss: 0.4441 - mean_squared_error: 0.4441
Epoch 8/10
2/2 [==============================] - 0s 12ms/step - loss: 0.3805 - mean_squared_error: 0.3805
Epoch 9/10
2/2 [==============================] - 0s 9ms/step - loss: 0.3790 - mean_squared_error: 0.3790
Epoch 10/10
2/2 [========================

In [4]:
res = model.evaluate()
print (res)

1/1 - 0s - loss: 0.8878 - mean_squared_error: 0.8878 - 159ms/epoch - 159ms/step
0.9422097160794695


In [5]:
# we predict with the last 7 days
model.predict(df[-7:])


1/1 [==============================] - 0s 146ms/step


array([ 0.0762028 , -0.12489999,  0.09852157], dtype=float32)

# Investigating the use of random forests for feature extraction

In [ ]:
# Lets run the model 10 times for temperature_celsius in Prague

df = pd.read_csv('docs/data/GlobalWeatherRepository.csv')

df = adjust_dataset(df, 'Prague')

target = 'temperature_celsius'
X_train, y_train, X_test, y_test = split_data(df, target, int(len(df) * 0.7))

results_without_rf = []
for i in range(10):
    model = Ann(y_train, y_test, 7, 3, target)
    model.build_model()
    res = model.evaluate()
    results_without_rf.append(res)
    

Epoch 1/50


1/1 [==============================] - 0s 426ms/step - loss: 29.5833 - mean_squared_error: 29.5833
Epoch 2/50
1/1 [==============================] - 0s 8ms/step - loss: 26.3258 - mean_squared_error: 26.3258
Epoch 3/50
1/1 [==============================] - 0s 7ms/step - loss: 24.3412 - mean_squared_error: 24.3412
Epoch 4/50
1/1 [==============================] - 0s 8ms/step - loss: 22.1720 - mean_squared_error: 22.1720
Epoch 5/50
1/1 [==============================] - 0s 8ms/step - loss: 21.0418 - mean_squared_error: 21.0418
Epoch 6/50
1/1 [==============================] - 0s 9ms/step - loss: 20.5321 - mean_squared_error: 20.5321
Epoch 7/50
1/1 [==============================] - 0s 8ms/step - loss: 20.8606 - mean_squared_error: 20.8606
Epoch 8/50
1/1 [==============================] - 0s 7ms/step - loss: 19.1847 - mean_squared_error: 19.1847
Epoch 9/50
1/1 [==============================] - 0s 10ms/step - loss: 18.5033 - mean_squared_error: 18.5033
Epoch 10/50
1/1 [===================

In [ ]:
# Time execution = 47.5 seconds

print (sum(results_without_rf)/len(results_without_rf))

5.698


In [ ]:
# Lets apply Random Forest

from sklearn.ensemble import RandomForestRegressor


model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

feature_importances = model.feature_importances_


threshold = 0.001  # threshold mean importance
selected_features = [feature for feature, importance in sorted(enumerate(feature_importances), key=lambda x: x[1], reverse=True) if importance > threshold]

X_selected_train = X_train.iloc[:, selected_features]



print (X_selected_train.columns)

Index(['feels_like_celsius', 'moonrise', 'moon_illumination',
       'moon_phase_Waxing Crescent', 'air_quality_PM2.5', 'humidity',
       'air_quality_Nitrogen_dioxide', 'air_quality_Ozone', 'gust_kph',
       'sunset', 'pressure_mb', 'sunrise', 'moonset', 'wind_kph',
       'air_quality_Sulphur_dioxide'],
      dtype='object')


In [ ]:
df = pd.read_csv('docs/data/GlobalWeatherRepository.csv')

df = adjust_dataset(df, 'Prague')

target = 'temperature_celsius'
X_train, y_train, X_test, y_test = split_data(df, target, int(len(df) * 0.7))

X_train = X_train.iloc[:, selected_features]
X_test = X_test.iloc[:, selected_features]

results_with_rf = []
for i in range(10):
    model = Ann(y_train, y_test, 7, 3, target)
    model.build_model()
    res = model.evaluate()
    results_with_rf.append(res)


Epoch 1/50
1/1 [==============================] - 0s 432ms/step - loss: 42.9931 - mean_squared_error: 42.9931
Epoch 2/50
1/1 [==============================] - 0s 8ms/step - loss: 35.5642 - mean_squared_error: 35.5642
Epoch 3/50
1/1 [==============================] - 0s 8ms/step - loss: 32.0271 - mean_squared_error: 32.0271
Epoch 4/50
1/1 [==============================] - 0s 9ms/step - loss: 30.9500 - mean_squared_error: 30.9500
Epoch 5/50
1/1 [==============================] - 0s 9ms/step - loss: 29.2369 - mean_squared_error: 29.2369
Epoch 6/50
1/1 [==============================] - 0s 7ms/step - loss: 27.6318 - mean_squared_error: 27.6318
Epoch 7/50
1/1 [==============================] - 0s 9ms/step - loss: 26.9572 - mean_squared_error: 26.9572
Epoch 8/50
1/1 [==============================] - 0s 8ms/step - loss: 27.7543 - mean_squared_error: 27.7543
Epoch 9/50
1/1 [==============================] - 0s 7ms/step - loss: 24.4831 - mean_squared_error: 24.4831
Epoch 10/50
1/1 [=========

In [ ]:
print (sum(results_with_rf)/len(results_with_rf))

5.662999999999999


In [ ]:
# CONCLUSION: Random Forest does not improve the results